In [ ]:
# First, register for CoinMarketCap and obtain your API key by visiting https://coinmarketcap.com/api/.

# Then, import necessary libraries
import pandas as pd
import os 
from time import time
from time import sleep
import csv
import seaborn as sns
import matplotlib.pyplot as plt

# This sets the maximum number of columns and rows to display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
# This code below  was obtained from the Python section of the Quick Start Guide in the CoinMarketCap API documentation.
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' # changed "sandbox-api.coinmarketcap.com" to "pro-api.coinmarketcap.com", since it says "All requests should target domain https://pro-api.coinmarketcap.com." in the authentication section
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '3bfc4e2b-e29e-4747-82c0-362cb29c3927', #'b54bcf4d-1bca-4e8e-9a24-22ff2c3d462c',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

# If you receive an "IOPub data rate exceeded" error, try running the following command in the anaconda powershell:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
type(data)

In [ ]:
# This normalizes the data and creates a DataFrame
df = pd.json_normalize(data['data'])
# Add a new column with the current timestamp
df['timestamp'] = pd.Timestamp.now()
df

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' # changed "sandbox-api.coinmarketcap.com" to "pro-api.coinmarketcap.com", since it says "All requests should target domain https://pro-api.coinmarketcap.com." in the authentication section
    parameters = {
      'start':'1',
      'limit':'100',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '3bfc4e2b-e29e-4747-82c0-362cb29c3927', #'b54bcf4d-1bca-4e8e-9a24-22ff2c3d462c',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.Timestamp.now()
    
    # Write the data to the CSV file
    with open('crypto_data.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        if file.tell() == 0: # If the file is empty, write the headers
            writer.writerow(df.columns)
        writer.writerows(df.values)
    
    # Read the data from the CSV file into a DataFrame
    df = pd.read_csv('crypto_data.csv')

In [ ]:
for i in range(333): # since you only have 333 runs per day for API key usage
    api_runner()
    print('API Runner completed')
    sleep(30) #sleep for 30 seconds

In [ ]:
results = pd.read_csv('crypto_data.csv')
results

In [ ]:
# Group the data by coin name and calculate the mean of the percent change columns
results2 = results.groupby('name', sort=False)[['quote.USD.percent_change_1h',
                                                'quote.USD.percent_change_24h',
                                                'quote.USD.percent_change_7d',
                                                'quote.USD.percent_change_30d',
                                                'quote.USD.percent_change_60d',
                                                'quote.USD.percent_change_90d']].mean()
results2

In [ ]:
# Stack the columns of the grouped dataframe to create a new dataframe
results3 = results2.stack()
results3

In [ ]:
type(results3)

In [ ]:
# Convert the new dataframe to a dataframe with a single column named "values"
results4 = results3.to_frame(name='values')
results4

In [ ]:
results4.count()

In [ ]:
# Reset the index of the new dataframe 
results5 = results4.reset_index()
results5

In [ ]:
# Rename the column named "level_1" to "percent_change" in the new dataframe
results6 = results5.rename(columns={'level_1': 'percent_change'})
results6

In [ ]:
# Replace the long percent change column names with shorter ones in the new dataframe
results6['percent_change'] = results6['percent_change'].replace(['quote.USD.percent_change_1h',
                                                                   'quote.USD.percent_change_24h',
                                                                   'quote.USD.percent_change_7d',
                                                                   'quote.USD.percent_change_30d',
                                                                   'quote.USD.percent_change_60d',
                                                                   'quote.USD.percent_change_90d'],
                                                                  ['1h','24h', '7d', '30d', '60d', '90d'])
results6

In [ ]:
# Create a categorical plot showing the relationship between percent change and coin name
sns.catplot(x='percent_change', y='values', hue='name', data=results6, kind='point')
plt.show()

In [ ]:
# Create a new dataframe with the desired columns and filter for Bitcoin data only
results7 = results[['name', 'quote.USD.price', 'timestamp']]
results7 = results7.query("name == 'Bitcoin'")
results7

In [ ]:
# Create a line plot showing Bitcoin's price over time
sns.set_theme(style="darkgrid")
ax = sns.lineplot(x='timestamp', y='quote.USD.price', data=results7)
ax.tick_params(axis='x', rotation=90)
plt.xticks(range(len(results7['timestamp'])), results7['timestamp'], fontsize=8)
plt.show()